In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image

torch.manual_seed(123)

## Class Dataset
inherit from torch.utils.data.Dataset. Will later be used for DataLoader.

In [22]:
# inherit the torch.utils.data.Dataset class
class Dataset(Dataset) :

    def __init__(self, image_dir, label_frame, transform=None) :
        """
        Args:
            image_dir (string): Directory with all the images.
            label_frame (string): Path to the csv file with class or regression labels.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.image_dir = image_dir
        self.label_frame = pd.read_csv(label_frame)
        self.transform = transform

    def __len__(self) :
        return self.label_frame.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_dir,
                                self.label_frame.iloc[idx, 0])
        images = Image.open(img_name)
        labels = self.label_frame.iloc[idx, 1:]
        labels = np.array([labels])
        labels = labels.astype('float').reshape(-1, 2)

        if self.transform:
            images = self.transform(images)

        return images, labels

## Using Dataset and DataLoader
**Setting up the directory to access data**
- root = "directory to the EAAI23 folder (the folder containing the folder data/train_set/images (or val_set/images for testing))"
- train_dir = root + "directory from EAAI23 to the images folder." (The .csv data file has the image name corresponding to the train/val set so all train and validation photos can be in the same folder.)
- train_frame = root + "data/train_reg.csv" (if pull and merge the new .csv data file, should not change this.)


**Transformation_fnc**
- Current set will do the following:
    1. Resize the photos from and size and shape to (128, 128)
    2. Convert the *PIL.Image* to *torch.tensor*
    3. Normalize the image data to the range [0, 1]
- Can customized by changing the transformations different *torchvision.transforms* functions

**Dataset**
- Should not change any arguments of this function

**DataLoader**
- only adjust the *batch_size* if needed. Currently set to 1024 data point per batch.

In [23]:
from torchvision import transforms

root = "C:/Phanh/BuAnhNet/EAAI23/"
train_dir = root + "data/train_set/images"
train_frame = root + "data/train_reg.csv"

# the mean and std was calculated using ImageNet dataset.
#   Do we want to recalculate these values to fit with our dataset ?
transform_fnc = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],             # might have to recalculate these values
        std=[0.229, 0.224, 0.225],              #   for our dataset. otherwise, do not change.
    ),
])

# data was pre-shuffled before saved to csv file
dataSet = Dataset(train_dir, train_frame, transform=transform_fnc)                      
# shuffle = False since the data has already been shuffled and splitted into the corresponding .csv file
dataLoader = torch.utils.data.DataLoader(dataSet, batch_size=1024, shuffle=False)       

## Implement DataLoader in training
DataLoader will return images and labels corresponding to batchX, batchY used in training
- The code following show the implementation of getting batchX, batchY for training.

In [25]:
for batchX, batchY in dataLoader :
    print(type(batchX), type(batchY))
    print(batchX.shape, batchY.shape)
    break

<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([1024, 3, 128, 128]) torch.Size([1024, 1, 2])
